In [ ]:
!pip install pydub
!pip install optuna
import librosa
import glob
from pydub import AudioSegment
from google.colab import drive
import os
from pydub import AudioSegment
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, accuracy_score, f1_score, make_scorer, confusion_matrix, log_loss, recall_score, precision_score
from sklearn.linear_model import LogisticRegression
import optuna
from sklearn.model_selection import cross_validate, StratifiedKFold
import pickle
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 KB 13.6 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=d3dc06a967d9c5f63fcc47af66a37ce636681d6c4f44c99a14fb5f15eb72e2d0
  Stored in directory: /root/.cache/pip

In [ ]:
drive.mount('/content/drive/', force_remount=True)
path = '/content/drive/MyDrive/PUM/projekt/low'
os.chdir(path)
!ls

Mounted at /content/drive/
100.wav  19.wav  28.wav  37.wav  46.wav  55.wav  64.wav  73.wav  82.wav  91.wav
10.wav	 1.wav	 29.wav  38.wav  47.wav  56.wav  65.wav  74.wav  83.wav  92.wav
11.wav	 20.wav  2.wav	 39.wav  48.wav  57.wav  66.wav  75.wav  84.wav  93.wav
12.wav	 21.wav  30.wav  3.wav	 49.wav  58.wav  67.wav  76.wav  85.wav  94.wav
13.wav	 22.wav  31.wav  40.wav  4.wav	 59.wav  68.wav  77.wav  86.wav  95.wav
14.wav	 23.wav  32.wav  41.wav  50.wav  5.wav	 69.wav  78.wav  87.wav  96.wav
15.wav	 24.wav  33.wav  42.wav  51.wav  60.wav  6.wav	 79.wav  88.wav  97.wav
16.wav	 25.wav  34.wav  43.wav  52.wav  61.wav  70.wav  7.wav	 89.wav  98.wav
17.wav	 26.wav  35.wav  44.wav  53.wav  62.wav  71.wav  80.wav  8.wav	 99.wav
18.wav	 27.wav  36.wav  45.wav  54.wav  63.wav  72.wav  81.wav  90.wav  9.wav


In [ ]:
filenames = list(os.listdir('/content/drive/MyDrive/PUM/projekt/low'))
LabelHorizontal = []
LabelVertical = []

with open("../motors_ground_truth") as file: 
  for line in file:
    line = line.strip().split('    ')
    #print(line)
    if float(line[1]) > -30 and float(line[1]) < 30 and f'{line[0]}.wav' in filenames:
      LabelVertical.append(1)
    if float(line[1]) > 30 and float(line[1]) < 90 and f'{line[0]}.wav' in filenames:
      LabelVertical.append(2)
    if float(line[1]) > 90 and float(line[1]) < 150 and f'{line[0]}.wav' in filenames:
      LabelVertical.append(3)
    if float(line[1]) > 150 and float(line[1]) < -150 and f'{line[0]}.wav' in filenames:
      LabelVertical.append(4)
    if float(line[1]) > -150 and float(line[1]) < -90 and f'{line[0]}.wav' in filenames:
      LabelVertical.append(5)
    if float(line[1]) > -90 and float(line[1]) < -30 and f'{line[0]}.wav' in filenames:
      LabelVertical.append(6)

    if float(line[2]) > -60 and float(line[2]) < -21 and f'{line[0]}.wav' in filenames:
      LabelHorizontal.append(1)
    if float(line[2]) > -20 and float(line[2]) < 20 and f'{line[0]}.wav' in filenames:
      LabelHorizontal.append(2)
    if float(line[2]) > 21 and float(line[2]) < 60 and f'{line[0]}.wav' in filenames:
      LabelHorizontal.append(3)
